In [2]:
import ethnicolr
from ethnicolr import census_ln, pred_census_ln
import pandas as pd
from genderize import Genderize
import re

In [4]:
#genderize = Genderize(api_key='') 
#use above if you have a Genderize API key
genderize = Genderize()

Race/Ethnicity groups follow census groups, trained from US 2010 Census data

Genderize is using social media data

Need to install using jupyterhub account - ethnicolr needs python 3.6

## Author Name created

In [114]:
data = pd.read_csv('https://raw.githubusercontent.com/NCBI-Codeathons/pubmed-codeathon-team1/main/data/out/pmid_data.csv')

In [115]:
data.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country'],
      dtype='object')

In [116]:
data.title.head()

0              Moyamoya disease and moyamoya syndrome.
1                        Progress in moyamoya disease.
2                                    Moyamoya Disease.
3     Moyamoya Disease: A Review of Clinical Research.
4    Etiology and pathogenesis of Moyamoya Disease:...
Name: title, dtype: object

In [117]:
data.authors.head()

0               Scott|R Michael|RM|;Smith|Edward R|ER|
1    Shang|Shuling|S|;Zhou|Da|D|;Ya|Jingyuan|J|;Li|...
2    Fujimura|Miki|M|;Bang|Oh Young|OY|;Kim|Jong S|JS|
3    Hishikawa|Tomohito|T|;Sugiu|Kenji|K|;Date|Isao|I|
4    Huang|Shuo|S|;Guo|Zhen-Ni|ZN|;Shi|Mingchao|M|;...
Name: authors, dtype: object

In [118]:
data.authors.tail()

7201    Khorramrouz|Fatemeh|F|;Doustmohammadian|Azam|A...
7202    Miciak|Jeremy|J|http://orcid.org/0000-0002-066...
7203    Kellner|Rebecca L|RL|;Agathis|Alexandra Z|AZ|;...
7204    Cornely|Ronald M|RM|;Subramanya|Vinita|V|;Owen...
7205    Jayaram|Lata|L|https://orcid.org/0000-0002-665...
Name: authors, dtype: object

In [5]:
# To split, spliting on ; will split by author, the | will split the names. With middle names also included, for first names will get rid of any single letter words

In [119]:
data.authors.str.split(';').str[-1].str.split('|').str[1].str.split(' ').str[0]

0       Edward
1          Ran
2         Jong
3         Isao
4       Mingli
         ...  
7201    Maryam
7202     David
7203     Brian
7204     Ambar
7205      Anne
Name: authors, Length: 7206, dtype: object

In [120]:
data.authors.str.split(';').str[-1].str.split('|').str[1].str.split(' ').str[1]

0           R
1         NaN
2           S
3         NaN
4         NaN
        ...  
7201      NaN
7202        J
7203        A
7204      NaN
7205    Marie
Name: authors, Length: 7206, dtype: object

In [121]:
data['LastAuthFirstName'] = data.authors.str.split(';').str[-1].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')

/tmp/ipykernel_267718/89657527.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['LastAuthFirstName'] = data.authors.str.split(';').str[-1].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')


In [122]:
data['LastAuthFirstName']

0           Edward
1              Ran
2             Jong
3             Isao
4           Mingli
           ...    
7201        Maryam
7202         David
7203         Brian
7204         Ambar
7205    Anne Marie
Name: LastAuthFirstName, Length: 7206, dtype: object

In [123]:
data.authors.str.split(';').str[-1].str.split('|').str[0]

0              Smith
1               Meng
2                Kim
3               Date
4                Rao
            ...     
7201        Khosravi
7202         Francis
7203         Coakley
7204    Kulshreshtha
7205       Southcott
Name: authors, Length: 7206, dtype: object

In [124]:
data['LastAuthLastName'] = data.authors.str.split(';').str[-1].str.split('|').str[0]

In [125]:
data.authors.str.split(';').str[0].str.split('|').str[0]

0             Scott
1             Shang
2          Fujimura
3         Hishikawa
4             Huang
           ...     
7201    Khorramrouz
7202         Miciak
7203        Kellner
7204        Cornely
7205        Jayaram
Name: authors, Length: 7206, dtype: object

In [126]:
data['FirstAuthLastName'] = data.authors.str.split(';').str[0].str.split('|').str[0]

In [130]:
data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')

/tmp/ipykernel_267718/2683773378.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')


0        Michael
1        Shuling
2           Miki
3       Tomohito
4           Shuo
          ...   
7201     Fatemeh
7202      Jeremy
7203     Rebecca
7204      Ronald
7205        Lata
Name: authors, Length: 7206, dtype: object

In [131]:
data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('((?<=^)|(?<= )).((?=$)|(?= ))', '')

/tmp/ipykernel_267718/1639744005.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('((?<=^)|(?<= )).((?=$)|(?= ))', '')


0        Michael
1        Shuling
2           Miki
3       Tomohito
4           Shuo
          ...   
7201     Fatemeh
7202      Jeremy
7203    Rebecca 
7204     Ronald 
7205        Lata
Name: authors, Length: 7206, dtype: object

In [132]:
data['FirstAuthFirstName'] = data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')

/tmp/ipykernel_267718/3602760542.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['FirstAuthFirstName'] = data.authors.str.split(';').str[0].str.split('|').str[1].str.replace('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '')


In [133]:
data.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName'],
      dtype='object')

In [134]:
df = data[['LastAuthFirstName','FirstAuthFirstName',]]

In [135]:
len(df)

7206

## Gender calculation

In [136]:
genderLAuthor = pd.DataFrame(genderize.get(df.LastAuthFirstName, ))

In [137]:
gender1Author = pd.DataFrame(genderize.get(df.FirstAuthFirstName, ))

In [138]:
genderLAuthor

,name,gender,probability,count,p_female,p_male
0,Edward,male,0.99,22248,0.01,0.99
1,Ran,male,0.58,1379,0.42,0.58
2,Jong,male,0.94,2523,0.06,0.94
3,Isao,male,0.99,326,0.01,0.99
4,Mingli,male,0.52,23,0.48,0.52
...,...,...,...,...,...,...
7201,Maryam,female,0.98,9767,0.98,0.02
7202,David,male,0.99,501011,0.01,0.99
7203,Brian,male,0.99,51483,0.01,0.99
7204,Ambar,female,0.84,748,0.84,0.16


In [139]:
gender1Author

,name,gender,probability,count
0,Michael,male,0.99,245870
1,Shuling,female,0.90,21
2,Miki,male,0.75,19891
3,Tomohito,male,1.00,71
4,Shuo,male,0.72,150
...,...,...,...,...
7201,Fatemeh,female,0.98,816
7202,Jeremy,male,0.99,48233
7203,Rebecca,female,0.98,36835
7204,Ronald,male,0.99,23684


In [140]:
gender1Author.loc[gender1Author['gender']=='female', 'p_female'] = gender1Author['probability']
gender1Author.loc[gender1Author['gender']=='female', 'p_male'] = 1 - gender1Author['probability']

gender1Author.loc[gender1Author['gender']=='male', 'p_female'] = 1 - gender1Author['probability']
gender1Author.loc[gender1Author['gender']=='male', 'p_male'] = gender1Author['probability']

In [141]:
genderLAuthor.loc[genderLAuthor['gender']=='female', 'p_female'] = genderLAuthor['probability']
genderLAuthor.loc[genderLAuthor['gender']=='female', 'p_male'] = 1 - genderLAuthor['probability']

genderLAuthor.loc[genderLAuthor['gender']=='male', 'p_female'] = 1 - genderLAuthor['probability']
genderLAuthor.loc[genderLAuthor['gender']=='male', 'p_male'] = genderLAuthor['probability']

In [142]:
gender1Author

,name,gender,probability,count,p_female,p_male
0,Michael,male,0.99,245870,0.01,0.99
1,Shuling,female,0.90,21,0.90,0.10
2,Miki,male,0.75,19891,0.25,0.75
3,Tomohito,male,1.00,71,0.00,1.00
4,Shuo,male,0.72,150,0.28,0.72
...,...,...,...,...,...,...
7201,Fatemeh,female,0.98,816,0.98,0.02
7202,Jeremy,male,0.99,48233,0.01,0.99
7203,Rebecca,female,0.98,36835,0.98,0.02
7204,Ronald,male,0.99,23684,0.01,0.99


In [143]:
gender1Author.gender.astype(str).value_counts()

male      3812
female    2592
None       802
Name: gender, dtype: int64

In [144]:
genderLAuthor.gender.astype(str).value_counts()

male      4730
female    2063
None       413
Name: gender, dtype: int64

In [50]:
gender1Author.columns

Index(['name', 'gender', 'probability', 'count', 'p_female', 'p_male'], dtype='object')

In [147]:
gender1Author = gender1Author.rename(columns = {'gender':'gender_first_author', 'p_female':'p_first_author_female', 'p_male':'p_first_author_male'})

In [148]:
genderLAuthor = genderLAuthor.rename(columns = {'gender':'gender_last_author', 'p_female':'p_last_author_female', 'p_male':'p_last_author_male'})

In [169]:
len(data)

7206

In [155]:
len(genderLAuthor)

7206

In [156]:
data.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName'],
      dtype='object')

In [176]:
data1 = data.join(genderLAuthor)

In [177]:
len(data1)

7206

In [179]:
data1 = data1.join(gender1Author, lsuffix='lastauthor', rsuffix='firstauthor')

In [180]:
len(data1)

7206

In [181]:
data1

,pmid,query,search_type,page,relative_citation_ratio,human,animal,molecular_cellular,apt,is_clinical,...,probabilitylastauthor,countlastauthor,p_last_author_female,p_last_author_male,namefirstauthor,gender_first_author,probabilityfirstauthor,countfirstauthor,p_first_author_female,p_first_author_male
0,19297575,Moyamoya,relevance,1,29.60,1.0,0.0,0.0,0.95,No,...,0.99,22248,0.01,0.99,Michael,male,0.99,245870,0.01,0.99
1,29911252,Moyamoya,relevance,1,9.28,1.0,0.0,0.0,0.95,No,...,0.58,1379,0.42,0.58,Shuling,female,0.90,21,0.90,0.10
2,27960175,Moyamoya,relevance,1,2.81,1.0,0.0,0.0,0.95,No,...,0.94,2523,0.06,0.94,Miki,male,0.75,19891,0.25,0.75
3,27549666,Moyamoya,relevance,1,2.39,1.0,0.0,0.0,0.75,No,...,0.99,326,0.01,0.99,Tomohito,male,1.00,71,0.00,1.00
4,28381201,Moyamoya,relevance,1,2.76,0.5,0.5,0.0,0.75,No,...,0.52,23,0.48,0.52,Shuo,male,0.72,150,0.28,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,33988094,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.25,No,...,0.98,9767,0.98,0.02,Fatemeh,female,0.98,816,0.98,0.02
7202,35608744,literacy skills,pubdate_desc,2,NaN,0.0,0.0,0.0,0.00,No,...,0.99,501011,0.01,0.99,Jeremy,male,0.99,48233,0.01,0.99
7203,35603494,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.00,No,...,0.99,51483,0.01,0.99,Rebecca,female,0.98,36835,0.98,0.02
7204,35597972,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.00,No,...,0.84,748,0.84,0.16,Ronald,male,0.99,23684,0.01,0.99


## Race Ethnicity

In [185]:
RaceEthGroup1st = {'pctwhite':'firstauthor_percent_white', 'pctblack':'firstauthor_percent_black', 'pctapi':'firstauthor_percent_asian', 'pctaian':'firstauthor_percent_american_indian_alaskan_native', 'pct2prace':'firstauthor_percent_multi_race', 'pcthispanic':'firstauthor_percent_hispanic'}

In [183]:
 dre = census_ln(data, 'FirstAuthLastName')

In [184]:
dre.head()

,pmid,query,search_type,page,relative_citation_ratio,human,animal,molecular_cellular,apt,is_clinical,...,LastAuthFirstName,LastAuthLastName,FirstAuthLastName,FirstAuthFirstName,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,19297575,Moyamoya,relevance,1,29.60,1.0,0.0,0.0,0.95,No,...,Edward,Smith,Scott,Michael,62.60,32.26,0.41,1.15,1.90,1.68
1,29911252,Moyamoya,relevance,1,9.28,1.0,0.0,0.0,0.95,No,...,Ran,Meng,Shang,Shuling,7.63,1.66,88.49,0.00,1.39,0.83
2,27960175,Moyamoya,relevance,1,2.81,1.0,0.0,0.0,0.95,No,...,Jong,Kim,Fujimura,Miki,6.05,(S),84.08,0.00,8.92,(S)
3,27549666,Moyamoya,relevance,1,2.39,1.0,0.0,0.0,0.75,No,...,Isao,Date,Hishikawa,Tomohito,NaN,NaN,NaN,NaN,NaN,NaN
4,28381201,Moyamoya,relevance,1,2.76,0.5,0.5,0.0,0.75,No,...,Mingli,Rao,Huang,Shuo,1.03,0.09,96.83,0.02,1.74,0.29


In [187]:
dre = dre.rename(columns = RaceEthGroup1st)


In [188]:
dre.head()

,pmid,query,search_type,page,relative_citation_ratio,human,animal,molecular_cellular,apt,is_clinical,...,LastAuthFirstName,LastAuthLastName,FirstAuthLastName,FirstAuthFirstName,firstauthor_percent_white,firstauthor_percent_black,firstauthor_percent_asian,firstauthor_percent_american_indian_alaskan_native,firstauthor_percent_multi_race,firstauthor_percent_hispanic
0,19297575,Moyamoya,relevance,1,29.60,1.0,0.0,0.0,0.95,No,...,Edward,Smith,Scott,Michael,62.60,32.26,0.41,1.15,1.90,1.68
1,29911252,Moyamoya,relevance,1,9.28,1.0,0.0,0.0,0.95,No,...,Ran,Meng,Shang,Shuling,7.63,1.66,88.49,0.00,1.39,0.83
2,27960175,Moyamoya,relevance,1,2.81,1.0,0.0,0.0,0.95,No,...,Jong,Kim,Fujimura,Miki,6.05,(S),84.08,0.00,8.92,(S)
3,27549666,Moyamoya,relevance,1,2.39,1.0,0.0,0.0,0.75,No,...,Isao,Date,Hishikawa,Tomohito,NaN,NaN,NaN,NaN,NaN,NaN
4,28381201,Moyamoya,relevance,1,2.76,0.5,0.5,0.0,0.75,No,...,Mingli,Rao,Huang,Shuo,1.03,0.09,96.83,0.02,1.74,0.29


In [189]:
RaceEthGroupLast = {'pctwhite':'lastauthor_percent_white', 'pctblack':'lastauthor_percent_black', 'pctapi':'lastauthor_percent_asian', 'pctaian':'lastauthor_percent_american_indian_alaskan_native', 'pct2prace':'lastauthor_percent_multi_race', 'pcthispanic':'lastauthor_percent_hispanic'}

In [190]:
dfre = census_ln(data, 'LastAuthLastName')

In [191]:
dfre = dfre.rename(columns = RaceEthGroupLast)

In [192]:
dfre.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'lastauthor_percent_white',
       'lastauthor_percent_black', 'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic'],
      dtype='object')

In [193]:
dfre = dfre[['lastauthor_percent_white',
       'lastauthor_percent_black', 'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic']]

In [194]:
dre = dre.join(dfre)

In [195]:
dre.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'firstauthor_percent_white',
       'firstauthor_percent_black', 'firstauthor_percent_asian',
       'firstauthor_percent_american_indian_alaskan_native',
       'firstauthor_percent_multi_race', 'firstauthor_percent_hispanic',
       'lastauthor_percent_white', 'lastauthor_percent_black',
       'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic'],
      dtype='object')

In [196]:
dre.tail()

,pmid,query,search_type,page,relative_citation_ratio,human,animal,molecular_cellular,apt,is_clinical,...,firstauthor_percent_asian,firstauthor_percent_american_indian_alaskan_native,firstauthor_percent_multi_race,firstauthor_percent_hispanic,lastauthor_percent_white,lastauthor_percent_black,lastauthor_percent_asian,lastauthor_percent_american_indian_alaskan_native,lastauthor_percent_multi_race,lastauthor_percent_hispanic
7201,33988094,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.25,No,...,NaN,NaN,NaN,NaN,78.80,(S),3.18,(S),13.78,3.53
7202,35608744,literacy skills,pubdate_desc,2,NaN,0.0,0.0,0.0,0.00,No,...,NaN,NaN,NaN,NaN,63.89,27.57,1.30,2.08,2.44,2.72
7203,35603494,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.00,No,...,0.88,0.14,0.99,1.38,78.90,17.68,0.48,0.29,1.39,1.26
7204,35597972,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.00,No,...,(S),(S),(S),(S),NaN,NaN,NaN,NaN,NaN,NaN
7205,35596530,literacy skills,pubdate_desc,2,NaN,1.0,0.0,0.0,0.00,No,...,89.22,0.00,(S),(S),93.55,0.00,(S),2.76,2.76,(S)


In [198]:
dre.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'firstauthor_percent_white',
       'firstauthor_percent_black', 'firstauthor_percent_asian',
       'firstauthor_percent_american_indian_alaskan_native',
       'firstauthor_percent_multi_race', 'firstauthor_percent_hispanic',
       'lastauthor_percent_white', 'lastauthor_percent_black',
       'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic'],
      dtype='object')

In [197]:
data1.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'namelastauthor', 'gender_last_author',
       'probabilitylastauthor', 'countlastauthor', 'p_last_author_female',
       'p_last_author_male', 'namefirstauthor', 'gender_first_author',
       'probabilityfirstauthor', 'countfirstauthor', 'p_first_author_female',
       'p_first_author_male'],
      dtype='object')

In [201]:
dre1 = dre[['firstauthor_percent_white',
       'firstauthor_percent_black', 'firstauthor_percent_asian',
       'firstauthor_percent_american_indian_alaskan_native',
       'firstauthor_percent_multi_race', 'firstauthor_percent_hispanic',
       'lastauthor_percent_white', 'lastauthor_percent_black',
       'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic']]

In [203]:
data1 = data1.join(dre1)

In [206]:
data1.columns

Index(['pmid', 'query', 'search_type', 'page', 'relative_citation_ratio',
       'human', 'animal', 'molecular_cellular', 'apt', 'is_clinical',
       'citation_count', 'cited_by', 'references', 'title', 'abstract',
       'journal', 'authors', 'affiliations', 'pubdate', 'mesh_terms',
       'publication_types', 'chemical_list', 'keywords', 'languages',
       'country', 'LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'namelastauthor', 'gender_last_author',
       'probabilitylastauthor', 'countlastauthor', 'p_last_author_female',
       'p_last_author_male', 'namefirstauthor', 'gender_first_author',
       'probabilityfirstauthor', 'countfirstauthor', 'p_first_author_female',
       'p_first_author_male', 'firstauthor_percent_white',
       'firstauthor_percent_black', 'firstauthor_percent_asian',
       'firstauthor_percent_american_indian_alaskan_native',
       'firstauthor_percent_multi_race', 'firstauthor_percent_hispanic',
       'lasta

In [207]:
data2 = data1[['pmid', 'query', 'search_type', 'page','LastAuthFirstName', 'LastAuthLastName', 'FirstAuthLastName',
       'FirstAuthFirstName', 'gender_last_author', 'p_last_author_female',
       'p_last_author_male', 'gender_first_author','p_first_author_female',
       'p_first_author_male', 'firstauthor_percent_white',
       'firstauthor_percent_black', 'firstauthor_percent_asian',
       'firstauthor_percent_american_indian_alaskan_native',
       'firstauthor_percent_multi_race', 'firstauthor_percent_hispanic',
       'lastauthor_percent_white', 'lastauthor_percent_black',
       'lastauthor_percent_asian',
       'lastauthor_percent_american_indian_alaskan_native',
       'lastauthor_percent_multi_race', 'lastauthor_percent_hispanic']]

In [215]:
data1.to_csv('RaceEthGenderFirstandLastAuthor.csv')

In [208]:
data2.to_csv('RaceEthGender.csv')